# Save Imspector measurement parameters as JSON

This notebook can be used to save the parameters of a measurement to JSON text files, so we can re-use them in automated measurements using a ```JSONSettingsLoader``` callback.

Some parameters (pixel size, FOV length) can be set from the automation code via dedicated callbacks, but in general, the idea is to save *template parameters* for all images you wish to take in an automation run (overviews, STED details, ...) and only change things like stage/scan position dynamically during the run.

So the following, among others, should be saved to a file and re-used:

* how many and which channels to use
* objective
* laser powers
* 3d STED setting (percent)
* dwell times and line accumulation
* RESCue
* ...

## Hints

* it's best to deactivate easyCommander before saving parameters
* pay attention to settings like "lock aspect ratio" in Imspector (making image size settable in only one dimension)
    * if a setting is grayed-out in the GUI, we can't change it from code either

## Basic saving

We want to save the whole Imspector parameter tree, which we can get as a dictionary via the ```.value_at('', ...)``` method.

Then, we can just save it using the ```json``` module:

In [ ]:
import json

import specpy as sp


# get current measurement parameters and hardware parameters
imspector = sp.get_application()
current_measurement_parameters = imspector.value_at('', sp.ValueTree.Measurement).get()
hardware_parameters = imspector.value_at('', sp.ValueTree.Hardware).get()

# dump to JSON
with open('parameters.json', 'w') as fd:
    json.dump(current_measurement_parameters, fd, indent=1)

## Batch Saving all parameters

In the code below, we can save the parameters of all configurations of a mesurement to multiple JSON files, with automatic naming.

Optionally, this also saves hardware settings (e.g. SLM calibration). **Currently, changing them during automation runs not necessary in most cases, only use it if you know what you are doing!**

First, we select what to save and where:

In [35]:
import json
import os
import warnings

import specpy as sp


# directory to save to
output_directory = 'config_json'

# base filename
output_filename = 'test2color'

# whether to save all configurations or only the currently selected one
save_all_configurations = True

# whether to also save hardware parameters
save_hardware_parameters = True

# Whether to overwrite existing files
overwrite_existing_files = False

Now, follow the steps below to save the parameters:

1. open existing or create **new measurement** in Imspector
2. set all parameters in Imspector
3. execute code below to save JSON file(s)

**make sure the correct window is selected in Imspector!**

In [ ]:
# make output directory absolute
output_directory = os.path.abspath(output_directory)

# make sure the directory to which we want to save exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# connect to Imspector & active measurement
imspector = sp.get_application()
ms = imspector.active_measurement()
conf_active = ms.active_configuration()

if save_all_configurations: confs_to_save = {name: ms.configuration(name) for name in ms.configuration_names()}
else: confs_to_save = {conf_active.name(): conf_active}

for name, conf in confs_to_save.items():

    # if we save multiple confs, add conf name to outfile
    if save_all_configurations: outfile = os.path.join(output_directory, output_filename + f'_{name}.json')
    else: outfile = os.path.join(output_directory, output_filename + '.json')

    if os.path.exists(outfile) and not overwrite_existing_files:
        warnings.warn(f'target file {outfile} exists, skipping.')
        continue

    # activate and get measurement parameters
    ms.activate(conf)
    parameters = imspector.value_at('', sp.ValueTree.Measurement).get()

    # dump to JSON
    with open(outfile, 'w') as fd:
        json.dump(parameters, fd, indent=1)

    print(f'saved measurement parameters of configuration {name} to {outfile}')

# re-activate the previously active configuration
ms.activate(conf_active)


if save_hardware_parameters:

    # get hardware parameters
    hardware_params = imspector.value_at('', sp.ValueTree.Hardware).get()

    outfile = os.path.join(output_directory, output_filename + '_hardware.json')
    if os.path.exists(outfile) and not overwrite_existing_files:
        warnings.warn(f'target file {outfile} exists, skipping.')
    else:
        with open(outfile, 'w') as fd:
            json.dump(hardware_params, fd, indent=1)
        print(f'saved hardware parameters to {outfile}')
